In [1]:
from langchain_openai import ChatOpenAI

# 配置你的自定义 API
llm = ChatOpenAI(
    model="deepseek-v3-0324",
    openai_api_base="https://api.juheai.top/v1",  # base_url
    openai_api_key="sk-w6Aw20y5ndUBd9FO6vOPkRCAuo1A5gNYdcz0X8HuB11bJ5h7",  # 你的 key
    temperature=0
)

# 测试调用
response = llm.invoke("你好呀！")
print(response.content)

你好呀！😊 很高兴见到你！今天有什么可以帮你的吗？无论是聊天、解答问题，还是需要一些建议，我都在这里哦～ （比如想聊聊日常、最新科技、或是帮忙查资料？）


In [2]:
import os
import requests
from typing import List
from langchain_community.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader


# 1. 自定义一个调用远程 API 的 Embedding 类
class RemoteQwenEmbeddings(Embeddings):
    def __init__(self, endpoint: str):
        self.endpoint = endpoint

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self._embed(t) for t in texts]

    def embed_query(self, text: str) -> List[float]:
        return self._embed(text)

    def _embed(self, text: str) -> List[float]:
        payload = {"texts": [text]}
        headers = {"Content-Type": "application/json"}
        response = requests.post(self.endpoint, json=payload, headers=headers)
        response.raise_for_status()
        data = response.json()
        # 假设返回格式是 {"data": [[向量]]}，你需要根据实际返回结构调整
        return data["embeddings"][0]


# 2. 初始化远程 embedding
endpoint = "https://gme-qwen2-vl-7b.ai4s.com.cn/embed/text"
embeddings = RemoteQwenEmbeddings(endpoint)

vectorstore_path = "faiss_index"

if os.path.isdir(vectorstore_path):
    print(f"发现已存在的向量库 '{vectorstore_path}'，正在加载...")
    vectorstore = FAISS.load_local(
        vectorstore_path,
        embeddings,
        allow_dangerous_deserialization=True
    )
    print("向量库加载完成。")
else:
    # 读取 PDF 并转换为 Document
    docs = []
    corpus_path = "./corpus"
    for file in os.listdir(corpus_path):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(corpus_path, file)
            loader = PyPDFLoader(pdf_path)
            pdf_docs = loader.load()
            docs.extend(pdf_docs)

    # 文本拆分
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    split_docs = text_splitter.split_documents(docs)

    # 构建向量库
    batch_size = 50
    vectorstore = None

    for i in range(0, len(split_docs), batch_size):
        batch_docs = split_docs[i:i + batch_size]

        if vectorstore is None:
            vectorstore = FAISS.from_documents(batch_docs, embeddings)
        else:
            vectorstore.add_documents(batch_docs)

    vectorstore.save_local(vectorstore_path)
    print("向量库构建完成，已保存到 faiss_index")


发现已存在的向量库 'faiss_index'，正在加载...
向量库加载完成。


In [2]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

# 本地模型路径
model_path = '/Users/sunforwing/Desktop/task/Qwen:Qwen3-Embedding-0.6B'

# 加载本地 embedding 模型
embeddings = HuggingFaceEmbeddings(model_name=model_path,model_kwargs={"device": "cpu"})

/var/folders/n6/lqm25gg94nn1d0dcsttg3y2w0000gn/T/ipykernel_53009/2437579702.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_path,model_kwargs={"device": "cpu"})


In [3]:
import os
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

# 本地 embedding 模型
#model_path = '/Users/sunforwing/Desktop/task/Qwen:Qwen3-Embedding-0.6B'
#embeddings = HuggingFaceEmbeddings(model_name=model_path)  # 可加 model_kwargs={"device": "cpu"}
vectorstore_path = "faiss_index"
if os.path.isdir(vectorstore_path):
    # 如果存在，则直接加载
    print(f"发现已存在的向量库 '{vectorstore_path}'，正在加载...")
    # 从本地加载时，需要提供 embedding 对象
    # 注意：新版 langchain 可能需要 allow_dangerous_deserialization=True
    vectorstore = FAISS.load_local(
        vectorstore_path, 
        embeddings,
        allow_dangerous_deserialization=True 
    )
    print("向量库加载完成。")
else:
    # 1. 读取 PDF 并转换为 Document
    docs = []
    corpus_path = "./corpus"
    for file in os.listdir(corpus_path):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(corpus_path, file)
            loader = PyPDFLoader(pdf_path)
            pdf_docs = loader.load()
            docs.extend(pdf_docs)
    
    # 2. 文本拆分
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    split_docs = text_splitter.split_documents(docs)
    
    # 3. 分批次向量化构建 FAISS
    batch_size = 50  # 每批处理 50 个 chunk
    vectorstore = None
    
    for i in range(0, len(split_docs), batch_size):
        batch_docs = split_docs[i:i+batch_size]
        
        if vectorstore is None:
            # 第一批创建 FAISS
            vectorstore = FAISS.from_documents(batch_docs, embeddings)
        else:
            # 后续批次添加，不需要传 embeddings
            vectorstore.add_documents(batch_docs)
    
    # 4. 保存向量库
    vectorstore.save_local("faiss_index")
    
    print("向量库构建完成，已保存到 faiss_index")


发现已存在的向量库 'faiss_index'，正在加载...
向量库加载完成。


In [4]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document


# ===== Dr.Hypothesis + RAG =====
def dr_hypothesis(patient_case, vectorstore: FAISS, top_k=5):
    # 从向量库检索相关文档
    docs = vectorstore.similarity_search(patient_case, k=top_k)
    context = "\n".join([doc.page_content for doc in docs])

    # 构造 prompt
    prompt = f"""
你是Dr.Hypothesis。根据患者病例信息，结合以下检索到的文献和指南内容，生成一个诊断假设列表：
病例信息：
{patient_case}

检索参考：
{context}

请列出可能的诊断假设（每个假设简短描述）。
"""
    chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt))
    return chain.run({})


In [5]:
# ===== Dr.Challenger =====
def dr_challenger(hypotheses):
    prompt = f"""
你是Dr.Challenger。分析以下诊断假设列表，指出可能的诊断错误，并提出合理替代诊断（如有）。
诊断假设列表：
{hypotheses}
"""
    chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt))
    return chain.run({})
    

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import json

def dr_clinical_reasoning(patient_case, hypotheses, challenged_hypotheses):

    # system 消息
    system_msg = SystemMessage(content=(
        "你是Dr.Clinical-Reasoning。根据患者完整病例信息、诊断假设以及Dr.Challenger的分析结果，生成最终诊断。"
        "输出必须严格遵循 JSON 结构，若字段信息没有，请使用空字符串。"
    ))

    # user 消息
    user_content = {
        "instruction": "请按照以下顺序输出最终诊断：主诊断（仅一个）、次要诊断（可多个）、鉴别诊断，并保留诊断依据。最终严格输出 JSON。",
        "json_template": {
            "患者信息": {"年龄":"","性别":"","入院日期":""},
            "临床表现": {"主诉":"","现病史":""},
            "病史信息": {"既往史":"","个人史":"","婚育史":"","家族史":""},
            "体格检查": "",
            "辅助检查": "",
            "诊断结果": {
                "主要诊断": {"名称":"","诊断依据":["依据1","依据2"]},
                "次要诊断":[{"名称":"次要诊断1","诊断依据":["依据1","依据2"]}],
                "鉴别诊断":["鉴别诊断1","鉴别诊断2"]
            },
            "治疗方案":["方案1","方案2"]
        },
        "病例信息": patient_case,
        "诊断假设": hypotheses,
        "Challenger分析": challenged_hypotheses
    }

    user_msg = HumanMessage(content=json.dumps(user_content, ensure_ascii=False, indent=2))

    # 直接调用 ChatOpenAI
    response = llm([system_msg, user_msg])

    # 返回文本
    return response.content


In [7]:
# ===== 示例调用 =====
input_data = {
  "medical record": "### 病史简介\n患者男性，52岁，因“反复腹痛、腹胀2周余”于2022-04-01收入消化科。\n\n现病史：患者2周余前无明显诱因反复出现腹痛、腹胀，为左下腹部持续性胀痛，与活动、体位无关，进食后可加重，排便、排气后可缓解，无恶心、呕吐，无反酸、烧心，无呕血、黑便，无胸闷、胸痛，无尿频、尿痛、尿黄，无发热、畏寒，自行服用“沉香化气胶囊、阿莫西林、西甲硅油、消化酶”等药物效果欠佳，上述症状反复发作，来我院急诊就诊。急诊化验检查：总胆红素51.92μmol/L；尿素氮15.99mmol/L；肌酐189μmol/L；尿酸621μmol/L；C反应蛋白138.6mg/L；D-二聚体69.29mg/L。2022-03-31腹部CT：下腔静脉-右颈内静脉人工血管转流术后所见；符合肝硬化、脾大、腹水、侧支循环形成表现；胆囊结石；胆囊壁略厚，胆囊炎不除外；考虑脾梗死可能性大，请结合临床；腹腔渗出性改变、积液。为进一步治疗收入消化科病房。\n\n既往史：高血压3年，血压最高可达160/100mmHg，现服用“氯沙坦钾”治疗，血压可控制在140/90mmHg。肾病综合征、阵发性睡眠性血红蛋白尿（paroxysmal nOCTurnal hemoglobinuria，PNH）2年，给予保肾药物（具体不详），定期返院治疗。丙型肝炎2年余，经正规抗丙肝治疗后复查阴性，否认结核、伤寒等其他传染病史及密切接触史。29年前因“布加综合征”行“下腔静脉-右颈内静脉人工血管转流术”，有同期输血史。\n\n个人史：无特殊，否认吸烟、饮酒史。\n\n婚育史：适龄结婚，育有1子。\n\n家族史：父亲因“心肌梗死”去世，母亲健在。有1弟弟，弟弟体健。否认家族遗传病史或重大疾病史。\n\n### 体格检查\n体温37.0℃，心率96次/分，呼吸20次/分，血压144/93mmHg。贫血貌，体型消瘦。右侧颈部可见一长约4cm手术瘢痕，腹部正中可见一长约15cm纵向手术瘢痕。双肺呼吸音粗，未闻及干湿性啰音。心前区无隆起及凹陷，心界无扩大。心率96次/分，节律规整，各瓣膜听诊区未闻及病理性杂音。腹部膨隆，腹壁张力大，左腹部压痛，有反跳痛。肝脾肋下未触及，Murphy’s征阴性，全腹叩诊鼓音，肝肾区无叩击痛，肠鸣音无亢进，移动性浊音阴性。双下肢轻度凹陷性水肿。\n\n### 辅助检查\n患者入院后化验血常规提示贫血（血红蛋白83g/L）、血小板减少（血小板71×10<sup>9</sup>/L），且合并感染（中性粒细胞百分比82.2%，C反应蛋白138.6mg/L）。肝肾功提示胆红素升高（总胆红素37.5μmol/L，直接胆红素27.2μmol/L），低蛋白血症（27.3g/L），肾功能不全（尿素氮15.3mmo/L，肌酐183μmol/L）。凝血常规提示凝血功能障碍，血浆凝血酶原时间（PT）延长（16.2秒）、部分活化凝血活酶时间（APTT）延长（43秒），D-二聚体显著升高（52.15mg/L）。"
}
patient_case = input_data["medical record"]
hypotheses = dr_hypothesis(patient_case, vectorstore)
# print("诊断假设:\n", hypotheses)
challenged = dr_challenger(hypotheses)
# print("\nChallenger分析:\n",challenged)
final_diagnosis = dr_clinical_reasoning(patient_case, hypotheses, challenged)
print("\n最终诊断:\n", final_diagnosis)

/var/folders/n6/lqm25gg94nn1d0dcsttg3y2w0000gn/T/ipykernel_53009/1487199304.py:25: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt))
/var/folders/n6/lqm25gg94nn1d0dcsttg3y2w0000gn/T/ipykernel_53009/1487199304.py:26: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return chain.run({})
/var/folders/n6/lqm25gg94nn1d0dcsttg3y2w0000gn/T/ipykernel_53009/2440300952.py:37: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([system_msg, user_msg])



最终诊断:
 ```json
{
  "患者信息": {
    "年龄": "52岁",
    "性别": "男性",
    "入院日期": "2022-04-01"
  },
  "临床表现": {
    "主诉": "反复腹痛、腹胀2周余",
    "现病史": "患者2周余前无明显诱因反复出现腹痛、腹胀，为左下腹部持续性胀痛，与活动、体位无关，进食后可加重，排便、排气后可缓解，无恶心、呕吐，无反酸、烧心，无呕血、黑便，无胸闷、胸痛，无尿频、尿痛、尿黄，无发热、畏寒，自行服用“沉香化气胶囊、阿莫西林、西甲硅油、消化酶”等药物效果欠佳。"
  },
  "病史信息": {
    "既往史": "高血压3年，肾病综合征、阵发性睡眠性血红蛋白尿（PNH）2年，丙型肝炎2年余；29年前因“布加综合征”行“下腔静脉-右颈内静脉人工血管转流术”，有同期输血史。",
    "个人史": "无特殊，否认吸烟、饮酒史。",
    "婚育史": "适龄结婚，育有1子。",
    "家族史": "父亲因“心肌梗死”去世，母亲健在。有1弟弟，弟弟体健。否认家族遗传病史或重大疾病史。"
  },
  "体格检查": "贫血貌，体型消瘦；右侧颈部可见一长约4cm手术瘢痕；腹部膨隆，腹壁张力大，左腹部压痛，有反跳痛；双下肢轻度凹陷性水肿。",
  "辅助检查": "贫血（血红蛋白83g/L）、血小板减少（血小板71×10^9/L）；感染标志物升高（中性粒细胞百分比82.2%，C反应蛋白138.6mg/L）；胆红素升高（总胆红素37.5μmol/L，直接胆红素27.2μmol/L）；低蛋白血症（27.3g/L）；肾功能不全（尿素氮15.3mmol/L，肌酐183μmol/L）；凝血功能障碍（PT 16.2秒，APTT 43秒）。",
  "诊断结果": {
    "主要诊断": {
      "名称": "阵发性睡眠性血红蛋白尿（PNH）相关血栓性微血管病（TMA）",
      "诊断依据": [
        "PNH病史、溶血性贫血（血红蛋白83g/L）、血小板减少、肾功能不全（尿素氮15.3mmol/L，肌酐183μmol/L）",
        "D-二聚体显著升高（52.15mg/L），提示微血栓形成",
        "临床表现（腹痛、腹胀、腹水）

In [8]:
def dr_clinical_reasoning(patient_case, hypotheses, challenged_hypotheses):
    system_msg = SystemMessage(content=(
        "你是Dr.Clinical-Reasoning。根据患者完整病例信息、诊断假设以及Dr.Challenger的分析结果，生成最终诊断。"
        "输出必须严格遵循 JSON 结构，若字段信息没有，请使用空字符串。"
    ))

    user_content = {
        "instruction": "请按照以下顺序输出最终诊断：主诊断（仅一个）、次要诊断（可多个）、鉴别诊断，并保留诊断依据。最终严格输出 JSON。",
        "json_template": {
            "患者信息": {"年龄":"","性别":"","入院日期":""},
            "临床表现": {"主诉":"","现病史":""},
            "病史信息": {"既往史":"","个人史":"","婚育史":"","家族史":""},
            "体格检查": "",
            "辅助检查": "",
            "诊断结果": {
                "主要诊断": {"名称":"","诊断依据":["依据1","依据2"]},
                "次要诊断":[{"名称":"次要诊断1","诊断依据":["依据1","依据2"]}],
                "鉴别诊断":["鉴别诊断1","鉴别诊断2"]
            },
            "治疗方案":["方案1","方案2"]
        },
        "病例信息": patient_case,
        "诊断假设": hypotheses,
        "Challenger分析": challenged_hypotheses
    }

    user_msg = HumanMessage(content=json.dumps(user_content, ensure_ascii=False, indent=2))

    final_text = ""
    # 用 llm.stream() 实时输出
    for chunk in llm.stream([system_msg, user_msg]):
        if chunk.content:
            print(chunk.content, end="", flush=True)  # 实时打印
            final_text += chunk.content              # 拼接完整结果

    return final_text  # 如果需要最后的完整 JSON，可以返回


In [9]:
# ===== 示例调用 =====
input_data = {
  "medical record": "### 病史简介\n患者男性，52岁，因“反复腹痛、腹胀2周余”于2022-04-01收入消化科。\n\n现病史：患者2周余前无明显诱因反复出现腹痛、腹胀，为左下腹部持续性胀痛，与活动、体位无关，进食后可加重，排便、排气后可缓解，无恶心、呕吐，无反酸、烧心，无呕血、黑便，无胸闷、胸痛，无尿频、尿痛、尿黄，无发热、畏寒，自行服用“沉香化气胶囊、阿莫西林、西甲硅油、消化酶”等药物效果欠佳，上述症状反复发作，来我院急诊就诊。急诊化验检查：总胆红素51.92μmol/L；尿素氮15.99mmol/L；肌酐189μmol/L；尿酸621μmol/L；C反应蛋白138.6mg/L；D-二聚体69.29mg/L。2022-03-31腹部CT：下腔静脉-右颈内静脉人工血管转流术后所见；符合肝硬化、脾大、腹水、侧支循环形成表现；胆囊结石；胆囊壁略厚，胆囊炎不除外；考虑脾梗死可能性大，请结合临床；腹腔渗出性改变、积液。为进一步治疗收入消化科病房。\n\n既往史：高血压3年，血压最高可达160/100mmHg，现服用“氯沙坦钾”治疗，血压可控制在140/90mmHg。肾病综合征、阵发性睡眠性血红蛋白尿（paroxysmal nOCTurnal hemoglobinuria，PNH）2年，给予保肾药物（具体不详），定期返院治疗。丙型肝炎2年余，经正规抗丙肝治疗后复查阴性，否认结核、伤寒等其他传染病史及密切接触史。29年前因“布加综合征”行“下腔静脉-右颈内静脉人工血管转流术”，有同期输血史。\n\n个人史：无特殊，否认吸烟、饮酒史。\n\n婚育史：适龄结婚，育有1子。\n\n家族史：父亲因“心肌梗死”去世，母亲健在。有1弟弟，弟弟体健。否认家族遗传病史或重大疾病史。\n\n### 体格检查\n体温37.0℃，心率96次/分，呼吸20次/分，血压144/93mmHg。贫血貌，体型消瘦。右侧颈部可见一长约4cm手术瘢痕，腹部正中可见一长约15cm纵向手术瘢痕。双肺呼吸音粗，未闻及干湿性啰音。心前区无隆起及凹陷，心界无扩大。心率96次/分，节律规整，各瓣膜听诊区未闻及病理性杂音。腹部膨隆，腹壁张力大，左腹部压痛，有反跳痛。肝脾肋下未触及，Murphy’s征阴性，全腹叩诊鼓音，肝肾区无叩击痛，肠鸣音无亢进，移动性浊音阴性。双下肢轻度凹陷性水肿。\n\n### 辅助检查\n患者入院后化验血常规提示贫血（血红蛋白83g/L）、血小板减少（血小板71×10<sup>9</sup>/L），且合并感染（中性粒细胞百分比82.2%，C反应蛋白138.6mg/L）。肝肾功提示胆红素升高（总胆红素37.5μmol/L，直接胆红素27.2μmol/L），低蛋白血症（27.3g/L），肾功能不全（尿素氮15.3mmo/L，肌酐183μmol/L）。凝血常规提示凝血功能障碍，血浆凝血酶原时间（PT）延长（16.2秒）、部分活化凝血活酶时间（APTT）延长（43秒），D-二聚体显著升高（52.15mg/L）。"
}
patient_case = input_data["medical record"]
hypotheses = dr_hypothesis(patient_case, vectorstore)
# print("诊断假设:\n", hypotheses)
challenged = dr_challenger(hypotheses)
# print("\nChallenger分析:\n",challenged)
final_diagnosis = dr_clinical_reasoning(patient_case, hypotheses, challenged)
# print("\n最终诊断:\n", final_diagnosis)

```json
{
  "患者信息": {
    "年龄": "52岁",
    "性别": "男",
    "入院日期": "2022-04-01"
  },
  "临床表现": {
    "主诉": "反复腹痛、腹胀2周余",
    "现病史": "患者2周余前无明显诱因出现左下腹部持续性胀痛，进食加重，排便排气后缓解，无恶心呕吐，无反酸烧心，无呕血黑便，无胸闷胸痛，无尿频尿痛尿黄，无发热畏寒，服药效果欠佳。"
  },
  "病史信息": {
    "既往史": "高血压3年，肾病综合征、阵发性睡眠性血红蛋白尿（PNH）2年，丙型肝炎2年余（治疗后阴性），29年前因布加综合征行下腔静脉-右颈内静脉人工血管转流术。",
    "个人史": "无吸烟饮酒史",
    "婚育史": "适龄结婚，育有1子",
    "家族史": "父亲心肌梗死去世，母亲健在，弟弟体健，无家族遗传病史"
  },
  "体格检查": "体温37.0℃，心率96次/分，血压144/93mmHg。贫血貌，消瘦。右侧颈部4cm手术瘢痕，腹部15cm纵向手术瘢痕。左腹部压痛伴反跳痛，腹部膨隆，移动性浊音阴性。双下肢轻度凹陷性水肿。",
  "辅助检查": "血常规：血红蛋白83g/L，血小板71×10⁹/L；炎症指标：中性粒细胞82.2%，CRP 138.6mg/L；肝肾功：总胆红素37.5μmol/L，白蛋白27.3g/L，尿素氮15.3mmol/L，肌酐183μmol/L；凝血功能：PT 16.2秒，APTT 43秒，D-二聚体52.15mg/L；CT：肝硬化、脾大、腹水、脾梗死可能。",
  "诊断结果": {
    "主要诊断": {
      "名称": "阵发性睡眠性血红蛋白尿（PNH）相关肠系膜静脉血栓",
      "诊断依据": [
        "PNH病史（高凝状态基础）",
        "剧烈腹痛+D-二聚体显著升高（52.15mg/L）",
        "CT提示血栓形成倾向（脾梗死可能）",
        "肝硬化门脉高压促进血栓形成"
      ]
    },
    "次要诊断": [
      {
        "名称": "肝硬化失代偿期",
        "诊断依据": [
          "丙肝+布加综合